# Create TROPESS AIRS-OMI Daily Plots

## Overview
This notebook will allow you to download [TROPESS](https://tes.jpl.nasa.gov/tropess) AIRS-OMI data and then use that data to create daily plots for a given date.  
The following suite of plots will be created from standard and summary data products:

| Species | Standard Product | Summary Product |
| :------ | :------ | :------ |
| Ozone (O3) | TRPSDL2O3AIRSOMIFS.1 | TRPSYL2O3AIRSOMIFS.1 |

In the table above, the "short name" of the data products is listed (e.g., "TRPSYL2O3AIRSOMIFS.1").  Short names are assigned by a NASA DAAC (in this case, the GES-DISC)
for each data products as a way to lookup or refer to a product with out using the product's full long name (e.g., "TROPESS AIRS-Aqua and OMI-Aura L2 Ozone for Reanalysis Stream, Summary Product V1").  You'll see these short names in the code below when there are blocks looping through the different products.

## Requirements

This notebook uses the [GES-DISC's](https://disc.gsfc.nasa.gov) simple subset service to retrieve requested data.  It requires a [NASA Earthdata login](https://urs.earthdata.nasa.gov/).  Please make sure you have one before attempting to run this notebook.  Also make sure you've setup a `.netrc` file in your home directory with your NASA Earthata login information.  You can use the follow steps to generate that file, if needed:

```sh
cd ~
touch .netrc
echo "machine urs.earthdata.nasa.gov login uid_goes_here password password_goes_here" > .netrc
chmod 0600 .netrc
```

## Import Libraries 

Standard (i.e., available through pip) libraries `datetime`, `requests`, `os`, and `glob` are imported below.  You will also need to clone or install the `tropessplots` library if you haven't already.  It is available at [https://github.com/NASA-TROPESS/tropessplots](https://github.com/NASA-TROPESS/tropessplots).

In [ ]:
import datetime as dt
import requests
import glob
import os

from tropessplots.io.airs_omi import read_l2summary, read_l2standard
from tropessplots.website_plots.airs_omi import plot_daily_overview

%load_ext autoreload
%autoreload 2

## Setup Global Variables

Please edit the variables in the next block as needed.  We'll do some checks and conversions after those variables are
set to reformat the date so we can use it in later blocks and to create any directories that don't exist.  We also have a 
list of all the data products we want to plot.

In [ ]:
# The date you want to plot in YYYYMMDD format.
DATE = '20250901'

# Where you want to store the data you download.
DOWNLOAD_DIRECTORY = '/tmp/download'

# Where you want to store the plot outputs.
PLOT_DIRECTORY = '/tmp/download'

# A list of the species you want to plot.  All species are listed here
# by default.  Edits to this list (additions or subtractions) will be 
# reflected in the SHORT_NAME_LIST variable below.  
SPECIES_ARRAY = ['O3']


In [ ]:
# The short names of the data products you want to plot.  These are automatically
# generated from SPECIES_ARRAY above.  Do not manually edit this list.
SHORT_NAME_LIST = []

if 'O3' in SPECIES_ARRAY:
    SHORT_NAME_LIST.append('TRPSDL2O3AIRSOMIFS.1')
    SHORT_NAME_LIST.append('TRPSYL2O3AIRSOMIFS.1')

# Formatting dates for use later
date_object = dt.datetime.strptime(DATE, '%Y%m%d')
iso_start_date = date_object.strftime('%Y-%m-%dT00:00:00.000Z')
iso_end_date = date_object.strftime('%Y-%m-%dT23:59:59.000Z')

# Create directories if they don't already exist
os.makedirs(DOWNLOAD_DIRECTORY, exist_ok=True)
os.makedirs(PLOT_DIRECTORY, exist_ok=True)

## Search for Data

Now that we are setup, we're going to start our work by downloading data from the GES-DISC.  We'll
make a request to the GES-DISC service to search for the data product and date we want.
Then we'll check to see if the serivce has found what we are looking for.  If not, we'll keep
waiting until it's found (if it takes too long though, we'll error out).  Once the service returns with a 
list of files we requested, we'll get a list of the URLs where the files are.

In [ ]:
files = []

for short_name in SHORT_NAME_LIST:
     # Send a request to the GES-DISC to search for files for the day
     # Again, you do need a NASA Earthdata account and a .netrc file in your home
     # directory to eventually download data found in these requests.
    r = requests.post('https://disc.gsfc.nasa.gov/service/subset/jsonwsp', 
                        json={"methodname": "subset", "args": {"role":"subset","start":iso_start_date, 
                                                            "end":iso_end_date,
                                                            "data":[{"datasetId":short_name.replace('.', '_')}]}}, 
                        headers={"Accept": "application/json, text/plain, */*", 
                                "Content-type": "application/json;charset=utf-8"})

    r.status_code
    info = r.json()
    status_counter = 0
    breaker_counter = 0

    # Get the status of the request; if not done, try again a few times
    while status_counter == 0:
        r = requests.post('https://disc.gsfc.nasa.gov/service/subset/jsonwsp', 
                    json={"methodname": "GetStatus", "args": {"jobId": info['result']['jobId'], "sessionId": info['result']['sessionId'] }, 
                                                                "type": "jsonwsp/request", "version": "1.0"}, 
                    headers={"Accept": "application/json, text/plain, */*", 
                            "Content-type": "application/json;charset=utf-8"})
        status_info = r.json()
        if status_info['result']['Status'] == 'Succeeded' and status_info['result']['PercentCompleted'] == 100:    
            status_counter = 1
        elif breaker_counter > 50:
            print('Not finding any files for this date %s' % DATE)
            break
        else:
            breaker_counter += 1

    r = requests.post('https://disc.gsfc.nasa.gov/service/subset/jsonwsp', 
                        json={"methodname": "GetResult", "args": {"jobId": info['result']['jobId'], "sessionId": info['result']['sessionId'] }, 
                                                                        "type": "jsonwsp/request", "version": "1.0"}, 
                        headers={"Accept": "application/json, text/plain, */*", 
                                    "Content-type": "application/json;charset=utf-8"})
    link_info = r.json()

    # Extract the individual URLs of each file
    for this_info in link_info['result']['items']:
        if short_name in this_info['label']:
            print('Found file %s' % this_info['link'])
            files.append(this_info['link'])


## Download the Data

Now that we have a list of URLs for the files we are interested in, we will download them
to our local system to the place we set in the DOWNLOAD_DIRECTORY.

In [ ]:
# Download each file locally.
for this_file in files:
    r = requests.get(this_file, allow_redirects=True)
    open(DOWNLOAD_DIRECTORY + '/' + this_file.split('/')[-1], 'wb').write(r.content)
    print('Downloaded %s' % this_file.split('/')[-1])

## Plot the Data

Based on the `SHORT_NAMES_LIST` global variable, we'll crete an array of species names that we're going to plot.  We'll use the standard files for the plotting function, along with the summary files if they exist for that specific species.

First, we'll find the standard and summary files on the local system.  Then we'll read the data with the readers we imported from the `tropessplots` library. Then, finally, we will plot the data, again with a plotting function from `tropessplots`.

In [ ]:
for species in SPECIES_ARRAY:    
    
    # Find files
    l2summary_file = glob.glob(DOWNLOAD_DIRECTORY + '/*AIRS*OMI*Summary*' + species+'*' + DATE + '*.nc')
    l2standard_file = glob.glob(DOWNLOAD_DIRECTORY + '/*AIRS*OMI*Standard*' + species + '*' + DATE + '*.nc')
    figure_file = PLOT_DIRECTORY + '/TROPESS_AIRS-OMI_' + species + '_' + DATE + '.png'
    
    # Read data
    l2summary = read_l2summary(files=l2summary_file,
                                   verbose=0)
    l2standard = read_l2standard(files=l2standard_file,
                                 verbose=0)
    
    # Run plotting routine
    plot_daily_overview(l2summary=l2summary,
                        l2standard=l2standard,
                        file_out=figure_file)
    print('Produced plot %s' % figure_file)